# Combining Figures

xarray-plotly provides helper functions to combine multiple figures:

- **`overlay_figures`**: Overlay traces on the same axes
- **`add_secondary_y`**: Plot with two independent y-axes

In [ ]:
import plotly.express as px
import xarray as xr

from xarray_plotly import add_secondary_y, config, overlay_figures, xpx

config.notebook()

## Load Sample Data

In [ ]:
# Stock prices
df_stocks = px.data.stocks().set_index("date")
df_stocks.index = df_stocks.index.astype("datetime64[ns]")

stocks = xr.DataArray(
    df_stocks.values,
    dims=["date", "company"],
    coords={"date": df_stocks.index, "company": df_stocks.columns.tolist()},
    name="price",
)

# Gapminder data (subset: a few countries)
df_gap = px.data.gapminder()
countries = ["United States", "China", "Germany", "Brazil"]
df_gap = df_gap[df_gap["country"].isin(countries)]

# Convert to xarray
gap_pop = df_gap.pivot(index="year", columns="country", values="pop")
gap_gdp = df_gap.pivot(index="year", columns="country", values="gdpPercap")
gap_life = df_gap.pivot(index="year", columns="country", values="lifeExp")

population = xr.DataArray(
    gap_pop.values,
    dims=["year", "country"],
    coords={"year": gap_pop.index.values, "country": gap_pop.columns.tolist()},
    name="Population",
)

gdp_per_capita = xr.DataArray(
    gap_gdp.values,
    dims=["year", "country"],
    coords={"year": gap_gdp.index.values, "country": gap_gdp.columns.tolist()},
    name="GDP per Capita",
)

life_expectancy = xr.DataArray(
    gap_life.values,
    dims=["year", "country"],
    coords={"year": gap_life.index.values, "country": gap_life.columns.tolist()},
    name="Life Expectancy",
)

## overlay_figures

Overlay multiple figures on the same axes. Useful for showing data with a trend line, moving average, or different visualizations of related data.

### Stock Price with Moving Average

In [ ]:
# Select one company
goog = stocks.sel(company="GOOG")

# Calculate 20-day moving average
goog_ma = goog.rolling(date=20, center=True).mean()
goog_ma.name = "20-day MA"

# Raw prices as scatter
price_fig = xpx(goog).scatter()
price_fig.update_traces(marker={"size": 4, "opacity": 0.5}, name="Daily Price")

# Moving average as line
ma_fig = xpx(goog_ma).line()
ma_fig.update_traces(line={"color": "red", "width": 3}, name="20-day MA")

combined = overlay_figures(price_fig, ma_fig)
combined.update_layout(title="GOOG: Daily Price with Moving Average")
combined

### Multiple Companies with Moving Averages

In [ ]:
# Select a few companies
subset = stocks.sel(company=["GOOG", "AAPL", "MSFT"])
subset_ma = subset.rolling(date=20, center=True).mean()

# Raw as scatter (faded)
raw_fig = xpx(subset).scatter()
raw_fig.update_traces(marker={"size": 3, "opacity": 0.3})

# MA as lines (bold)
ma_fig = xpx(subset_ma).line()
ma_fig.update_traces(line={"width": 3})

combined = overlay_figures(raw_fig, ma_fig)
combined.update_layout(title="Tech Stocks: Raw Prices + Moving Averages")
combined

### With Facets

`overlay_figures` works with faceted figures as long as both have the same structure.

In [ ]:
# Faceted by company
raw_faceted = xpx(subset).scatter(facet_col="company")
raw_faceted.update_traces(marker={"size": 3, "opacity": 0.4})

ma_faceted = xpx(subset_ma).line(facet_col="company")
ma_faceted.update_traces(line={"color": "red", "width": 2})

combined = overlay_figures(raw_faceted, ma_faceted)
combined.update_layout(title="Faceted: Price + Moving Average per Company")
combined

## add_secondary_y

Plot two variables with different scales using independent y-axes. Essential when values have different magnitudes (e.g., population in millions vs GDP in thousands).

### Population vs GDP per Capita

In [ ]:
# Select one country
us_pop = population.sel(country="United States")
us_gdp = gdp_per_capita.sel(country="United States")

pop_fig = xpx(us_pop).bar()
pop_fig.update_traces(marker={"color": "steelblue", "opacity": 0.7}, name="Population")

gdp_fig = xpx(us_gdp).line()
gdp_fig.update_traces(line={"color": "red", "width": 3}, name="GDP per Capita")

combined = add_secondary_y(pop_fig, gdp_fig, secondary_y_title="GDP per Capita ($)")
combined.update_layout(
    title="United States: Population vs GDP per Capita",
    yaxis_title="Population",
)
combined

### Life Expectancy vs GDP

In [ ]:
china_life = life_expectancy.sel(country="China")
china_gdp = gdp_per_capita.sel(country="China")

life_fig = xpx(china_life).line()
life_fig.update_traces(line={"color": "green", "width": 3}, name="Life Expectancy")

gdp_fig = xpx(china_gdp).line()
gdp_fig.update_traces(line={"color": "orange", "width": 3, "dash": "dash"}, name="GDP per Capita")

combined = add_secondary_y(life_fig, gdp_fig, secondary_y_title="GDP per Capita ($)")
combined.update_layout(
    title="China: Life Expectancy vs GDP per Capita",
    yaxis_title="Life Expectancy (years)",
    legend={"orientation": "h", "y": 1.1},
)
combined

### Comparing Scales: Why Secondary Y-Axis Matters

Without secondary y-axis, one variable dominates:

In [ ]:
# Same data on single y-axis (population dwarfs GDP)
pop_fig = xpx(us_pop).line()
pop_fig.update_traces(name="Population")

gdp_fig = xpx(us_gdp).line()
gdp_fig.update_traces(name="GDP per Capita")

# With overlay_figures (same y-axis) - GDP looks flat
bad = overlay_figures(pop_fig, gdp_fig)
bad.update_layout(title="Same Y-Axis: GDP appears flat (scale mismatch)")
bad

In [ ]:
# With add_secondary_y - both variables visible
pop_fig = xpx(us_pop).line()
pop_fig.update_traces(name="Population", line={"color": "blue"})

gdp_fig = xpx(us_gdp).line()
gdp_fig.update_traces(name="GDP per Capita", line={"color": "red"})

good = add_secondary_y(pop_fig, gdp_fig, secondary_y_title="GDP ($)")
good.update_layout(title="Dual Y-Axis: Both variables clearly visible")
good

## Limitations

### overlay_figures
- Overlay must have same or fewer subplots than base
- Animation frames must match (or overlay must be static)

### add_secondary_y
- Does not support faceted figures (subplots)
- Animation frames must match (or secondary must be static)